# Installing Library

In [ ]:
# Install necessary libraries
!pip install librosa pydub pyAudioAnalysis hmmlearn eyed3
!pip install pyAudioAnalysis==0.3.14
!pip show pyAudioAnalysis
!pip install webrtcvad subprocess.run
!pip install soundfile

# **Import libraries**



In [ ]:
import os
import librosa
import soundfile as sf


# **Load file**

In [ ]:
# Load image from local machine
from google.colab import files

uploaded = files.upload()

Saving sample_3.wav to sample_3.wav


# **Chunk Audio**

In [ ]:
from pydub import AudioSegment, silence
import os
# Create a directory for saving chunked audio if it doesn't exist
output_dir = "./source/chunk_audio"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Load your audio file
audio_file = "./data/sample_3.wav"  # Replace with your actual file name
sound = AudioSegment.from_file(audio_file)

# Define silence threshold in dBFS
silence_thresh = -40

# Detect non-silent chunks with a minimum length of 850 ms (0.85 seconds)
non_silent_chunks = silence.detect_nonsilent(sound, min_silence_len=1000, silence_thresh=silence_thresh)

# Extract non-silent segments from the audio
for i, (start, end) in enumerate(non_silent_chunks):
    chunk = sound[start:end]
    chunk.export(os.path.join(output_dir, f"chunk_{i}.wav"), format="wav")

# Filter audio by male and female

In [ ]:
import os
import soundfile as sf
import librosa
import numpy as np
from sklearn.cluster import DBSCAN

# Load the audio file
audio_path = "./data/sample_3.wav"
y, sr = librosa.load(audio_path)

# Extract features (MFCC, energy)
mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
energy = librosa.feature.rms(y=y)

# Combine features
features = np.vstack([mfcc, energy])

# Transpose feature matrix
features = features.T

# Normalize features
features_norm = (features - features.mean(axis=0)) / features.std(axis=0)

# Define a range of eps values to experiment with
eps_values = [0.1, 0.5, 1.0, 1.5, 2.0]

# Iterate over eps values and perform clustering
for eps in eps_values:
    # Perform clustering (DBSCAN)
    dbscan = DBSCAN(eps=eps, min_samples=5)  # Adjust min_samples as needed
    cluster_labels = dbscan.fit_predict(features_norm)

    # Create directories to save segmented audio
    output_dir_male = "./source/male_segments"
    output_dir_female = "./source/female_segments"
    os.makedirs(output_dir_male, exist_ok=True)
    os.makedirs(output_dir_female, exist_ok=True)

    # Save segments to respective directories
    for i, label in enumerate(cluster_labels):
        segment_start = librosa.frames_to_time(i, sr=sr)
        segment_end = librosa.frames_to_time(i + 1, sr=sr) if i < len(cluster_labels) - 1 else librosa.get_duration(y=y, sr=sr)
        segment = y[int(segment_start * sr):int(segment_end * sr)]
        if len(segment) > 0:  # Check if segment has non-zero length
            if label == 0:
                sf.write(os.path.join(output_dir_male, f"male_segment_{i+1}.wav"), segment, sr)
            elif label == 1:
                sf.write(os.path.join(output_dir_female, f"female_segment_{i+1}.wav"), segment, sr)


# **Read Audio**

In [ ]:
from IPython.display import Audio
import os

# Directory containing the audio files
# audio_dir = 'female_audio_chunks'
audio_dir = './source/female_segments'

def extract_number(filename):
    # Assuming the numeric part is at the beginning of the filename before any non-digit characters
    return int(''.join(filter(str.isdigit, filename)))


# List all audio files in the directory
audio_files = [f for f in os.listdir(audio_dir) if f.endswith('.wav')]
# Sort the audio files using the custom key function
audio_files.sort(key=extract_number) # Sort the audio files alphabetically

# Iterate over each audio file and play it
for audio_file in audio_files:
    audio_path = os.path.join(audio_dir, audio_file)
    print(f"Playing audio file: {audio_file}")
    display(Audio(audio_path))


Playing audio file: female_segment_5.wav


Playing audio file: female_segment_6.wav


Playing audio file: female_segment_7.wav


Playing audio file: female_segment_8.wav


Playing audio file: female_segment_9.wav


...

Playing audio file: female_segment_382.wav


Playing audio file: female_segment_4571.wav


Playing audio file: female_segment_4572.wav


Playing audio file: female_segment_4573.wav


Playing audio file: female_segment_4574.wav


# Delete Audio

In [ ]:
import os

# Directory containing the files you want to delete
# directory_path = 'male_audio_chunks'
directory_path = './source/male_segments_eps_1.0'
# List all files and directories in the directory
files = os.listdir(directory_path)

# Iterate over each item and delete it if it's a file
for item in files:
    file_path = os.path.join(directory_path, item)
    if os.path.isfile(file_path):
        os.remove(file_path)
    else:
        print(f"Skipping directory: {item}")

print("All files in the directory have been deleted.")

All files in the directory have been deleted.


# Download files

In [ ]:
import shutil
from google.colab import files

# Zip the directory named 'chunk_audio'
shutil.make_archive('chunk_audio', 'zip', 'chunk_audio')

# Download the zipped directory
files.download('chunk_audio.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>